Objetivo: Encontrar regiões mais adensadas e maior renda per capita em São Paulo ou outros municípios a fim de buscar maior quantidade de corridas premium como motorista de aplicativo.

A demanda por corridas premium em aplicativos como Uber e 99 pode ser influenciada por dois fatores principais: a densidade populacional e a renda per capita. É possível supor que, em áreas mais densamente povoadas, haja uma maior quantidade de pessoas se deslocando com frequência, o que tende a gerar mais solicitações de transporte por aplicativo. Nessas regiões, como em centros urbanos e bairros com alta concentração de habitantes, a lógica sugere que o uso desse tipo de serviço seja mais comum devido ao grande número de potenciais usuários.

Por outro lado, em locais onde a renda per capita é mais elevada, é razoável imaginar que o interesse por categorias premium, como Uber Black, seja maior. Isso se baseia na suposição de que pessoas com maior poder aquisitivo podem preferir serviços que ofereçam mais conforto e exclusividade, mesmo com tarifas mais altas. Assim, serviços que oferecem veículos de qualidade superior e motoristas com melhores qualificações podem atrair um público mais exigente, disposto a pagar por essa experiência.

Da mesma forma, categorias semi-premium, como Uber Comfort e 99 Plus, poderiam ser mais populares em regiões de alta renda, pois oferecem uma combinação de conforto a um custo relativamente mais acessível. Com base nisso, regiões de maior densidade populacional e renda mais alta poderiam ter uma demanda proporcionalmente maior por essas corridas premium e semi-premium, refletindo o perfil dos habitantes que vivem ou trabalham nessas localidades.

1. Obter os dados no IBGE do Censo;
2. Exploração dos dados obtido pelo censo;
3. Tratamento dos dados com os cálculos necessários para visualização;
4. Preparação da visualizações necessárias para identificação das 
   regiões com maiores densidades e renda.

1. Obter os dados no IBGE do Censo

    Hoje, nós estamos em 2024 e o censo de 2022 saiu preliminarmente contendo apenas os dados populacionais. No entanto, já que o estudo envolve população e renda, será utilizado os dados de 2010 que estão completos. Sabe-se que em 14 anos, teremos diferença nos dados, mas ao analisar o aumento populacional em São Paulo e São Bernardo do Campo, 1,7% e 5,9% respectivamente, de 2010 a 2022, provavelmente, não obtém-se grandes diferenças habitacionais no período. No entanto, a atualização deste estudo se torna peremptória para a assertividade do objetivo estipulado assim que divulgado os dados do Censo de 2022.

    O que são os setores censitários?

        "Os setores censitários são a menor porção de área utilizadas pelo IBGE para planejar, coletar e disseminar os resultados dos Censos e Pesquisas Estatísticas. 
         Esses setores são definidos a partir de diversos critérios:
            * A área e o número de domicílios do setor devem ser passíveis de serem percorridos e visitados por um único recenseador em tempo hábil.
            * O recorte do setor deve ser representativo em relação à população local.
            * O setor deve ter um número de domicílios suficiente para que seus dados agregados preservem o sigilo dos moradores"
            (G1 - https://g1.globo.com/economia/censo/noticia/2024/03/21/o-que-e-o-setor-censitario-medida-usada-pelo-ibge-para-fazer-o-censo.ghtml)

    Dados:
    
        Diretório do censo de todos os estados do Brasil por setor censitário:
            https://ftp.ibge.gov.br/Censos/Censo_Demografico_2010/Resultados_do_Universo/Agregados_por_Setores_Censitarios/
    
        Diretório dos arquivos geográficos dos setores censitários:
            https://geoftp.ibge.gov.br/organizacao_do_territorio/malhas_territoriais/malhas_de_setores_censitarios__divisoes_intramunicipais/censo_2010/setores_censitarios_shp/

Importação de bibliotecas

In [39]:
import os # Criação de estrutura de pastas e manuseamento de arquivos
from ftplib import FTP # Biblioteca para download dos arquivos de servidor ftp
from zipfile import ZipFile # Biblioteca para descompactar os arquivos
import pandas as pd # Tratamento dos dados 
import geopandas as gpd # Tratamento dos dados geográficos
import matplotlib.pyplot as plt # Biblioteca para visualização gráfica dos dados
import folium

Estrutura de pastas

In [40]:
parent_dir = os.getcwd() # Identificação do diretório de trabalho

"""
Criação dos diretórios para arquivamento dos arquivos
data: Arquivos baixados e a serem utilizados para tratamento.
artifacts: Todos os arquivos criados para utilização e visualização.
"""

directories = ["data", "artifacts"] 
for directory in directories:
    path = os.path.join(parent_dir, directory)
    try:
        os.mkdir(path)
    except FileExistsError:
        print("Directory already created!")
        
data_dir = os.path.join(parent_dir, directories[0])
artifacts_dir = os.path.join(parent_dir, directories[1])

Directory already created!
Directory already created!


Download de arquivos

Acessando os servidores FTP e baixando os arquivos.

In [41]:
ftp_server = "ftp.ibge.gov.br" # Servidor FTP do IBGE
ftp_dir = "Censos/Censo_Demografico_2010/Resultados_do_Universo/"\
    "Agregados_por_Setores_Censitarios/" # Diretório a ser acessado
# O servidor aceita acesso público, portanto user e password desnecessários
user = "anonymous" # Inserir anonymous para entrar no servidor
passwd = ""

ftp = FTP(ftp_server, user, passwd) # Configuração da variável de acesso ao servidor
ftp.cwd(ftp_dir) # Acessando o diretório escolhido
files = ftp.nlst() # variável contendo o conteúdo do diretório

ftp.dir() # Listagem do conteúdo do diretório

-rwxrwxr-x    1 ftp      ftp          1979 Oct 30  2023 1_Atualizacoes_20231030.txt
-rwxrwxr-x    1 ftp      ftp       5579523 Oct 30  2023 AC_20231030.zip
-rwxrwxr-x    1 ftp      ftp      22399914 Oct 30  2023 AL_20231030.zip
-rwxrwxr-x    1 ftp      ftp      31064700 Oct 30  2023 AM_20231030.zip
-rwxrwxr-x    1 ftp      ftp       5194028 Oct 30  2023 AP_20231030.zip
-rwxrwxr-x    1 ftp      ftp      126886126 Oct 30  2023 BA_20231030.zip
-rwxrwxr-x    1 ftp      ftp      71781739 Oct 30  2023 CE_20231030.zip
-rwxrwxr-x    1 ftp      ftp      21585996 Oct 30  2023 DF_20231030.zip
-rwxrwxr-x    1 ftp      ftp      34133643 Oct 30  2023 Documentacao_Agregado_dos_Setores_2010_20231030.zip
-rwxrwxr-x    1 ftp      ftp      37084676 Oct 30  2023 ES_20231030.zip
-rwxrwxr-x    1 ftp      ftp      53664403 Oct 30  2023 GO_20231030.zip
-rwxrwxr-x    1 ftp      ftp      48421505 Oct 30  2023 MA_20231030.zip
-rwxrwxr-x    1 ftp      ftp      169948641 Oct 30  2023 MG_20231030.zip
-rwxrwxr-x    

Nota-se que temos 3 tipos de arquivos:
  * 1 arquivo texto
  * 1 arquivo de documentação compactado
  * Arquivos do censo de cada estado compactado

In [42]:
docum = "Documentacao_Agregado" # variável para obter o arquivo de documentação
UFs = ["SP"] # lsita dos estados desejados
files_check = [ filename for filename in os.listdir(data_dir) ] # lista dos arquivos no diretório 

# Loop para download dos arquivos no servidor FTP
for file in files:
        if (file.startswith(docum) or file[:2] in UFs) and (file not in files_check):
            with open(os.path.join(data_dir,file), 'wb') as fp:
                ftp.retrbinary('RETR ' + file, fp.write)
                print(file+" downloaded to "+data_dir)

ftp.quit() # fechamento da conexão com o servidor FTP

'221 Goodbye.'

In [50]:
# Acessando o servidor ftp com as informações geográficas dos setores censitários
ftp_server = "geoftp.ibge.gov.br"
ftp_dir = "organizacao_do_territorio/malhas_territoriais/"\
    "malhas_de_setores_censitarios__divisoes_intramunicipais/censo_2010/"\
    "setores_censitarios_shp/"
user = "anonymous"
passwd = ""

ftp = FTP(ftp_server, user, passwd)
ftp.cwd(ftp_dir)
ftp.dir()

drwxrwxr-x    2 ftp      ftp          4096 Jun 14  2016 1_leia_me
drwxrwxr-x    2 ftp      ftp          4096 Jun 14  2016 ac
drwxrwxr-x    2 ftp      ftp          4096 Jun 14  2016 al
drwxrwxr-x    2 ftp      ftp          4096 Jun 14  2016 am
drwxrwxr-x    2 ftp      ftp          4096 Jun 14  2016 ap
drwxrwxr-x    2 ftp      ftp          4096 Jun 14  2016 ba
drwxrwxr-x    2 ftp      ftp          4096 Jun 14  2016 ce
drwxrwxr-x    2 ftp      ftp          4096 Jun 14  2016 df
drwxrwxr-x    2 ftp      ftp          4096 Jun 14  2016 es
drwxrwxr-x    2 ftp      ftp          4096 Jun 14  2016 go
drwxrwxr-x    2 ftp      ftp          4096 Jun 14  2016 ma
drwxrwxr-x    2 ftp      ftp          4096 Jun 14  2016 mg
drwxrwxr-x    2 ftp      ftp          4096 Jun 14  2016 ms
drwxrwxr-x    2 ftp      ftp          4096 Jun 14  2016 mt
drwxrwxr-x    2 ftp      ftp          4096 Jun 14  2016 pa
drwxrwxr-x    2 ftp      ftp          4096 Jun 14  2016 pb
drwxrwxr-x    2 ftp      ftp          4096 Jun 14

In [47]:
ftp.dir('sp')

-rw-rw-r--    1 ftp      ftp      11116004 Jun 14  2016 sp_distritos.zip
-rw-rw-r--    1 ftp      ftp       9387313 Jun 14  2016 sp_municipios.zip
-rwxrwxr-x    1 ftp      ftp      45523805 Nov 04  2020 sp_setores_censitarios.zip
-rw-rw-r--    1 ftp      ftp      11108462 Jun 14  2016 sp_subdistritos.zip


Verifica-se que dentro do diretório há quatro arquivos com as divisões dos municípios, distritos, subdistritos e setores censitários. Neste momento, deseja-se os setores censitários.

In [51]:
UFs_geo = ['sp'] # lista dos estados
extension_geo = "_setores_censitarios.zip" # variável com a extensão dos arquivos a serem baixados
files_check = [ filename for filename in os.listdir(data_dir) ] 

# Loop entre todos os diretórios dos estados para download dos arquivos geográficos
for UF_geo in UFs_geo:
    ftp.cwd(UF_geo) 
    files = ftp.nlst()
    for file in files:
        if (file == UF_geo+extension_geo) and (file not in files_check):
            with open(os.path.join(data_dir,file), 'wb') as fp:
                ftp.retrbinary('RETR ' + file, fp.write)
                print(file+" downloaded to "+data_dir)
    ftp.cwd("..") # Voltando ao diretório anterior para repetição do processo
ftp.quit()

'221 Goodbye.'

Extração dos arquivos compactados.

In [52]:
data_dir_files = os.listdir(data_dir) # Lista dos arquivos no diretório 'data'
data_dir_files

['Documentacao_Agregado_dos_Setores_2010_20231030.zip',
 'SP_Capital_20231030.zip',
 'SP_Exceto_Capital_20231030.zip',
 'sp_setores_censitarios.zip']

In [ ]:
for file in data_dir_files:
    try:
        if file.split(".")[1] == "zip":
            file_zipped = ZipFile(os.path.join(data_dir,file))
            
            new_folder_path = os.path.join(data_dir, file.split(".zip")[0])
            
            if not os.path.exists(new_folder_path):
                file_zipped.extractall(new_folder_path)
            
            file_zipped.close()
    except:
        print(f"{file} - Not a zipped file")

In [ ]:
files_list =[]

for dirpath, dirnames, filenames in os.walk(data_dir):
    for filename in filenames:
        file_path = os.path.join(dirpath, filename)
        files_list.append(file_path)
        
files_list

Tratamento de dados

In [ ]:
files_list_df =pd.DataFrame(files_list, columns=['filepaths'])
files_list_df

In [14]:
file_str_earning = 'Basico_' + 'SP1'
file_str_pop = 'pessoa13_' + 'sp1'

In [15]:
get_path_earning = files_list_df['filepaths'].str.contains(file_str_earning)
get_path_pop = files_list_df['filepaths'].str.contains(file_str_pop)

In [ ]:
file_path_earning = files_list_df.loc[get_path_earning, 'filepaths'].values[0]
print(file_path_earning)
file_path_pop = files_list_df.loc[get_path_pop,'filepaths'].values[0]
print(file_path_pop)

In [ ]:
city_earning = pd.read_csv(file_path_earning, encoding='iso-8859-1', sep=';')
city_earning.head().T

In [ ]:
city_earning.info()

In [ ]:
city_earning.isnull().sum()

In [ ]:
stg_city_earning = city_earning[['Cod_setor','Nome_do_municipio','Nome_do_distrito','Nome_do_bairro', 'V009']].reset_index(drop=True).copy()
stg_city_earning.head()

In [ ]:
stg_city_earning['Nome_do_municipio'].unique()

In [ ]:
city_pop = pd.read_csv(file_path_pop, encoding='iso-8859-1', sep=';')
city_pop.head()

In [ ]:
city_pop.info()

In [ ]:
city_pop.isnull().sum()

In [ ]:
null_values = city_pop.isnull().sum()
null_values[null_values >= 1]

In [26]:
stg_city_pop = city_pop[['Cod_setor','V001']].reset_index(drop=True).copy()

In [ ]:
stg_city_earning.info()

In [ ]:
stg_city_pop.info()

In [29]:
stg_city_pop['Cod_setor'] = stg_city_pop['Cod_setor'].astype('str')
stg_city_earning['Cod_setor'] = stg_city_earning['Cod_setor'].astype('str')

In [ ]:
stg_city = pd.merge(stg_city_earning, stg_city_pop, on='Cod_setor')
stg_city.head()

In [ ]:
stg_city.info()

In [ ]:
stg_city['V009'] = stg_city['V009'].str.replace(",", ".")
stg_city.head(10)

In [33]:
stg_city['V009'] = stg_city['V009'].astype('float64')

In [34]:
stg_city.rename(columns={'V001':'pop', 'V009':'renda'}, inplace=True)

In [ ]:
stg_city.head()

In [36]:
mask = stg_city['Nome_do_municipio'].str.contains("SÃO PAULO")
stg_city = stg_city[mask].copy()

In [ ]:
stg_city.head().T

In [ ]:
stg_city['Nome_do_distrito'].unique()

Tratamento de dados geográficos

In [ ]:
file_str_geo = '35SEE250GC_SIR.shp'
get_path_geo = files_list_df['filepaths'].str.contains(file_str_geo)
file_path_geo = files_list_df.loc[get_path_geo, 'filepaths'].values[0]
print(file_path_geo)

In [40]:
cities_setores_censitarios = gpd.read_file(file_path_geo)

In [ ]:
cities_setores_censitarios.head().T

In [42]:
stg_cities_setores_censitarios = cities_setores_censitarios.copy()

In [ ]:
stg_cities_setores_censitarios.head(10).T

In [ ]:
stg_cities_setores_censitarios.info()

In [ ]:
stg_cities_setores_censitarios.isnull().sum()

In [ ]:
stg_cities_setores_censitarios['NM_MUNICIP'].unique()

In [ ]:
mask = stg_cities_setores_censitarios['NM_MUNICIP'].str.contains('SÃO PAULO', case=False)
stg_city_setores_censitarios = stg_cities_setores_censitarios[mask].reset_index(drop=True).copy()
stg_city_setores_censitarios.head().T

In [ ]:
stg_city_setores_censitarios.info()

In [ ]:
stg_city_setores_censitarios.rename(columns={'CD_GEOCODI':'Cod_setor'}, inplace=True)
stg_city_setores_censitarios.head().T

In [ ]:
stg_city_setores_censitarios = stg_city_setores_censitarios.to_crs(epsg=31983)
stg_city_setores_censitarios['geometry'].head()

In [51]:
stg_city_setores_censitarios['area_ha'] = stg_city_setores_censitarios['geometry'].area / 1e4

In [52]:
stg_city_setores_censitarios = stg_city_setores_censitarios.to_crs(epsg=4326)

In [ ]:
stg_city_setores_censitarios.head().T

In [54]:
stg_city_setores_censitarios['Cod_setor'] = stg_city_setores_censitarios['Cod_setor'].astype('str')

Exploração de dados

In [ ]:
stg_city_setores_censitarios['Cod_setor']

In [ ]:
stg_city['Cod_setor']

In [ ]:
diff_in_series = set(stg_city_setores_censitarios['Cod_setor']).difference(stg_city['Cod_setor'])
diff_in_series

In [ ]:
stg_city_setores_censitarios[stg_city_setores_censitarios['Cod_setor'].str.contains('355030803000065')]

In [ ]:
stg_city.head().T

In [ ]:
stg_city.info()

In [ ]:
stg_city.describe()

stg_city[stg_city['Nome_do_distrito'].str.contains('ANHANGUERA')][60:70]

stg_city_setores_censitarios[stg_city_setores_censitarios['NM_DISTRIT'].str.contains('ANHANGUERA')][70:80].T

In [62]:
stg1_city_setores_censitarios = stg_city_setores_censitarios[['Cod_setor', 'geometry', 'area_ha']].copy()

In [ ]:
stg1_city_setores_censitarios.head().T

In [64]:
stg_city = pd.merge(stg_city, stg1_city_setores_censitarios, on='Cod_setor')

In [ ]:
stg_city.head().T

In [ ]:
stg_city.info()

In [ ]:
stg_city.isnull().sum()

In [68]:
stg_city['dens_pop'] = stg_city['pop'] / stg_city['area_ha']

In [ ]:
stg_city.head().T

In [ ]:
bairros =  stg_city.groupby('Nome_do_distrito').agg({'renda':'mean', 'pop':'sum', 'area_ha':'sum'})
bairros['dens_pop'] = bairros['pop'] / bairros['area_ha']
bairros

In [ ]:
bairros.describe()

In [ ]:
q3_bairros_renda = bairros['renda'].quantile(0.75)
print(q3_bairros_renda)

q1_bairros_dens = bairros['dens_pop'].quantile(0.25)
print(q1_bairros_dens)

mask = (bairros['renda']>=q3_bairros_renda) & (bairros['dens_pop']>=q1_bairros_dens)

bairros[mask].sort_values('renda', ascending=False)

In [ ]:
plt.scatter(bairros['dens_pop'], bairros['renda'])

plt.show()

Visualização de dados

In [ ]:
plt.figure(figsize=(15,5))

plt.subplot(1,2,1)
plt.hist(stg_city['renda'], bins=30, edgecolor='k')
plt.xlabel('R$ per capita')
plt.ylabel('Frequencia')

plt.subplot(1,2,2)
stg_city.boxplot(column='renda')
plt.ylabel('R$ per capita')

plt.show()

In [ ]:
plt.figure(figsize=(15,5))

plt.subplot(1,2,1)
plt.hist(stg_city['pop'], bins=30, edgecolor='k')
plt.xlabel('População - hab')
plt.ylabel('Frequencia')

plt.subplot(1,2,2)
stg_city.boxplot(column='pop')
plt.ylabel('População')

plt.show()

In [ ]:
plt.figure(figsize=(15,5))

plt.subplot(1,2,1)
plt.hist(stg_city['dens_pop'], bins=30, edgecolor='k')
plt.xlabel('Densidade populacional - hab/ha')
plt.ylabel('Frequencia')

plt.subplot(1,2,2)
stg_city.boxplot(column='dens_pop')
plt.ylabel('Densidade populacional - hab/ha')

plt.show()

In [ ]:
stg_city.describe()

In [ ]:
q3_renda = stg_city['renda'].quantile(0.75)
q3_dens_pop = stg_city['dens_pop'].quantile(0.50)

mask = (stg_city['renda'] >= q3_renda) & (stg_city['dens_pop'] >= q3_dens_pop)
top_25_stg_city = stg_city[mask].copy()
top_25_stg_city.head().T

In [ ]:
top_25_stg_city.info()

In [ ]:
top_25_stg_city.describe()

Visualização geográfica

file_str_geo = '35DSE250GC_SIR.shp'
get_path_geo = files_list_df['filepaths'].str.contains(file_str_geo)
file_path_geo = files_list_df.loc[get_path_geo, 'filepaths'].values[0]
print(file_path_geo)

cities_distritos = gpd.read_file(file_path_geo)

cities_distritos.head().T

cities_distritos.info()

cities_distritos[cities_distritos['CD_GEOCODD'].str.contains('355030804')]['geometry']

In [92]:
top_25_stg_city_gdf = gpd.GeoDataFrame(top_25_stg_city, geometry='geometry')

In [ ]:
hotels_list_path = os.path.join(data_dir, 'hotels_list.csv')

hotels_list_df = pd.read_csv(hotels_list_path)

hotels_list_df

In [94]:
map = folium.Map(location=[-23.5633, -46.66744],
                 tiles='Cartodb Positron',
                 zoom_start=13)

borders_style = {
    'color': 'green',
    'weight': 1,
    'fillColor': 'green',
    'fillOpacity': 0.15,
}

top_25 = folium.GeoJson(data=top_25_stg_city_gdf,
                             name='Top 25% renda e densidade populacional',
                             style_function=lambda x: borders_style,
                             )

top_25.add_to(map)

for index, row in hotels_list_df.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=row['name'],
    ).add_to(map)

map_dir = os.path.join(artifacts_dir, "top_25_sao_paulo.html")

map.save(map_dir)